<a href="https://colab.research.google.com/github/gilgarad/nlp_nlu/blob/master/code_tests/LSTM_Sequence_to_sequence_copied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Path to the data txt file on disk.
data_path = '/content/gdrive/My Drive/ai_data/kor-eng/kor.txt'

In [10]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
print('Number of lines:', len(lines))
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of lines: 800
Number of samples: 799
Number of unique input tokens: 69
Number of unique output tokens: 632
Max sequence length for inputs: 124
Max sequence length for outputs: 54


In [19]:
target_characters

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 'A',
 'B',
 'H',
 'M',
 'T',
 'a',
 'd',
 'h',
 'i',
 'm',
 'o',
 'p',
 'r',
 't',
 'y',
 '가',
 '각',
 '간',
 '갈',
 '감',
 '갑',
 '값',
 '갔',
 '강',
 '같',
 '개',
 '걀',
 '걔',
 '거',
 '걱',
 '건',
 '걸',
 '검',
 '겁',
 '것',
 '게',
 '겠',
 '겨',
 '격',
 '견',
 '결',
 '겼',
 '경',
 '계',
 '고',
 '곤',
 '곧',
 '곱',
 '곳',
 '공',
 '과',
 '관',
 '괜',
 '괴',
 '교',
 '구',
 '국',
 '군',
 '굳',
 '궁',
 '권',
 '귀',
 '그',
 '극',
 '근',
 '글',
 '금',
 '급',
 '기',
 '긴',
 '길',
 '까',
 '깎',
 '깔',
 '꺼',
 '꺾',
 '께',
 '꼈',
 '꼴',
 '꽃',
 '꽉',
 '꽤',
 '꾸',
 '꿨',
 '끄',
 '끈',
 '끊',
 '끌',
 '끓',
 '끔',
 '끝',
 '끼',
 '나',
 '난',
 '날',
 '남',
 '납',
 '났',
 '내',
 '낼',
 '냄',
 '냈',
 '냉',
 '냐',
 '냥',
 '너',
 '넌',
 '네',
 '녀',
 '녁',
 '년',
 '념',
 '녕',
 '노',
 '놀',
 '농',
 '놓',
 '놔',
 '놨',
 '누',
 '눈',
 '느',
 '는',
 '늘',
 '늙',
 '능',
 '늦',
 '니',
 '님',
 '다',
 '닥',
 '단',
 '닫',
 '달',
 '닮',
 '담',
 '답',
 '당',
 '대',
 '댔',
 '더',
 '덕',
 '던',
 '데',
 '덴',
 '

In [11]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Train on 639 samples, validate on 160 samples
Epoch 1/50
639/639 [==============================] - 4s 7ms/step - loss: 1.5862 - val_loss: 2.4038
Epoch 2/50
639/639 [==============================] - 3s 4ms/step - loss: 1.2724 - val_loss: 2.0994
Epoch 3/50
639/639 [==============================] - 3s 4ms/step - loss: 1.1537 - val_loss: 2.0450
Epoch 4/50
639/639 [==============================] - 3s 4ms/step - loss: 1.1239 - val_loss: 2.0514
Epoch 5/50
639/639 [==============================] - 3s 4ms/step - loss: 1.1117 - val_loss: 2.0426
Epoch 6/50
639/639 [==============================] - 3s 4ms/step - loss: 1.1035 - val_loss: 2.0378
Epoch 7/50
639/639 [==============================] - 3s 4ms/step - loss: 1.0962 - val_loss: 2.0457
Epoch 8/50
639/639 [==============================] - 3s 4ms/step - loss: 1.0915 - val_loss: 2.0500
Epoch 9/50
639/639 [==============================] - 3s 4ms/step - loss: 1.0869 - val_loss: 2.0390
Epoch 10/50
639/639 [==============================] -

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


-
Input sentence: Who?
Decoded sentence: 톰은       어.

-
Input sentence: Hello!
Decoded sentence: 톰은       어.

-
Input sentence: No way!
Decoded sentence: 톰은       어.

-
Input sentence: No way!
Decoded sentence: 톰은       어.

-
Input sentence: I'm sad.
Decoded sentence: 톰은       어.

-
Input sentence: Me, too.
Decoded sentence: 톰은       어.

-
Input sentence: Perfect!
Decoded sentence: 톰은       어.

-
Input sentence: Shut up!
Decoded sentence: 톰은       어.

-
Input sentence: Welcome.
Decoded sentence: 톰은       어.

-
Input sentence: Welcome.
Decoded sentence: 톰은       어.

-
Input sentence: Cheer up!
Decoded sentence: 톰은       어.

-
Input sentence: Get lost.
Decoded sentence: 톰은       어.

-
Input sentence: I'm ugly.
Decoded sentence: 톰은       어.

-
Input sentence: It hurts.
Decoded sentence: 톰은       어.

-
Input sentence: Let's go!
Decoded sentence: 톰은       어.

-
Input sentence: Don't lie.
Decoded sentence: 톰은       어.

-
Input sentence: Don't lie.
Decoded sentence: 톰은       어.

-
Input sente